<a href="https://colab.research.google.com/github/truongthuanr/self-project/blob/dev/11_Named_Entity_Recognition/Named_Entity_Recognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:

%%capture
!pip3 install datasets


In [3]:
import numpy as np
from keras.preprocessing.sequence import pad_sequences
from datasets import load_dataset
import os


In [66]:
from keras.models import Model
from keras.layers import Input,Embedding,TimeDistributed,\
                         Dropout,Conv1D,MaxPooling1D,\
                         Flatten,Bidirectional,LSTM,Dense,\
                         concatenate

from keras.initializers import RandomUniform

from keras.optimizers import Adam





In [49]:
# For vetorization
max_word_tokens = 24000
max_sentence_length = 30


# Dataset

Used dataset:

https://huggingface.co/datasets/conll2003

In [5]:
conll_data = load_dataset("conll2003")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

In [6]:
type(conll_data['train']['tokens'][0])

list

In [26]:
from keras.preprocessing.text import Tokenizer

In [22]:
tokenizer = Tokenizer(num_words=max_word_tokens)
tokenizer.fit_on_texts(conll_data['train']['tokens'])

## Embedding

In [11]:
def export_to_file(export_file_path, data):
    with open(export_file_path, "w") as f:
        for record in data:
            ner_tags = record["ner_tags"]
            tokens = record["tokens"]
            if len(tokens) > 0:
                f.write(
                    str(len(tokens))
                    + "\t"
                    + "\t".join(tokens)
                    + "\t"
                    + "\t".join(map(str, ner_tags))
                    + "\n"
                )


os.mkdir("data")
export_to_file("./data/conll_train.txt", conll_data["train"])
export_to_file("./data/conll_val.txt", conll_data["validation"])

# Data preprocessing


In [ ]:
# Data
wordEmbeddings = []
caseEmbeddings = []

# Model

In [29]:
from keras.layers import TextVectorization

In [ ]:
# Configuration



In [50]:
text_vectorizer = TextVectorization(max_tokens=max_word_tokens,   # how many word in the vocab lib
                                    standardize='lower_and_strip_punctuation', # process the tab
                                    split='whitespace', # split the token
                                    ngrams=None,
                                    output_mode='int', # map token to number
                                    output_sequence_length=max_sentence_length,
                                    pad_to_max_tokens=False,
                                    vocabulary=None,)
text_vectorizer.adapt(np.array([ " ".join(row) for row in conll_data['train']['tokens']]))

In [54]:
# np.array([ " ".join(row) for row in conll_data['train']['tokens'][:2]])

In [57]:
print(text_vectorizer.vocabulary_size())
print(text_vectorizer.get_vocabulary()[:10])

20070
['', '[UNK]', 'the', 'of', 'in', 'to', 'a', 'and', 'on', 'said']


In [59]:
## Embeding

embedding = Embedding(input_dim=max_word_tokens,
                      output_dim=128,
                      embeddings_initializer="uniform",
                      input_length=max_sentence_length)


In [70]:
# Word level
word_input = Input(shape=(1,),dtype="string",name='word_input')

# Tokenize
words = text_vectorizer(word_input)
# Embedding
words = embedding(words)
# Additional work feature

# CNN-extracted character feature


combined = words
# Bi-LSTM
output = Bidirectional(LSTM(200, return_sequences=True, dropout=0.50, recurrent_dropout=0.25))(combined)
# Output -> Tag Scoring -> Tag
output = Dense(1, activation="softmax")(output)



model_1 = Model(word_input, output, name="dense_model")



In [71]:
model_1.compile(loss="binary_crossentropy",
                optimizer=Adam(),
                metrics=["accuracy"])

In [72]:
model_1.summary()

Model: "dense_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 word_input (InputLayer)     [(None, 1)]               0         
                                                                 
 text_vectorization_3 (Text  (None, 30)                0         
 Vectorization)                                                  
                                                                 
 embedding (Embedding)       (None, 30, 128)           3072000   
                                                                 
 bidirectional_2 (Bidirecti  (None, 30, 400)           526400    
 onal)                                                           
                                                                 
 dense_2 (Dense)             (None, 30, 1)             401       
                                                                 
Total params: 3598801 (13.73 MB)
Trainable params: 3598

In [ ]:
# Charater level

# Character Embedding

# Additional character feature

# Convolution

# Max

# CNN extracted character feature


In [7]:
# Word level
words_input = Input(shape=(None,),dtype='int32',name='words_input')
words = Embedding(input_dim=wordEmbeddings.shape[0], output_dim=wordEmbeddings.shape[1],  weights=[wordEmbeddings], trainable=False)(words_input)
casing_input = Input(shape=(None,), dtype='int32', name='casing_input')
casing = Embedding(output_dim=caseEmbeddings.shape[1], input_dim=caseEmbeddings.shape[0], weights=[caseEmbeddings], trainable=False)(casing_input)

# Character level
character_input=Input(shape=(None,52,),name='char_input')
embed_char_out=TimeDistributed(Embedding(len(char2Idx),30,embeddings_initializer=RandomUniform(minval=-0.5, maxval=0.5)), name='char_embedding')(character_input)
dropout= Dropout(0.5)(embed_char_out)
conv1d_out= TimeDistributed(Conv1D(kernel_size=3, filters=30, padding='same',activation='tanh', strides=1))(dropout)
maxpool_out=TimeDistributed(MaxPooling1D(52))(conv1d_out)
char = TimeDistributed(Flatten())(maxpool_out)
char = Dropout(0.5)(char)



output = concatenate([words, casing,char])
output = Bidirectional(LSTM(200, return_sequences=True, dropout=0.50, recurrent_dropout=0.25))(output)
output = TimeDistributed(Dense(len(label2Idx), activation='softmax'))(output)

model = Model(inputs=[words_input, casing_input,character_input], outputs=[output])
model.compile(loss='sparse_categorical_crossentropy', optimizer='nadam')
model.summary()


# References

https://keras.io/examples/nlp/ner_transformers/